<a href="https://colab.research.google.com/github/uc2045/genai-projects/blob/master/Oct20%20WS%20LC2_Prompt_Engineering_with_Local_Open_Source_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<center> <h1> Prompt Engineering with Local Open Source LLMs</h1> </center>

<p style="margin-bottom:1cm;"></p>

_____

___[Created By: Dipanjan (DJ)](https://www.linkedin.com/in/dipanjans/)___

In this notebook we will learn how to download and run LLMs locally using this colab notebook. You can run this notebook in your local server also as long as you have a valid GPU with enough Memory to run these models!

The model we will be trying here is the:

__[Google Gemma 2B IT LLM](https://huggingface.co/google/recurrentgemma-2b-it)__ model which is a 2B parameter transformer LLM built by Google and is a instruct fine-tuned version of the [Google Gemma 2B LLM](https://huggingface.co/google/recurrentgemma-2b)

RecurrentGemma is a family of open language models built on a novel recurrent architecture developed at Google. Both pre-trained and instruction-tuned versions are available in English.

Like Gemma, RecurrentGemma models are well-suited for a variety of text generation tasks, including question answering, summarization, and reasoning. Because of its novel architecture, RecurrentGemma requires less memory than Gemma and achieves faster inference when generating long sequences.

__You will need at least 5GB of GPU memory to swiftly run inference with Recurrent Gemma IT 2B.__


When using Google Colab remember to change the runtime type as follows and select an available GPU to run the LLM faster

![](https://i.imgur.com/a26Qmdw.png)

## Check your GPU Memory Available

In [ ]:
!nvidia-smi

Sun Oct 20 16:24:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8              13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Install Necessary Dependencies

In [ ]:
!pip install transformers accelerate

__Restart the runtime from the Runtime menu above to make sure the installed libraries are ready to be used in Colab__

## Login to Huggingface using your Token

Get your token [here](https://huggingface.co/settings/tokens) and login using the following code

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Load the LLM locally using Huggingface

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model_id = "google/recurrentgemma-2b-it"
dtype = torch.bfloat16
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype=dtype,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/42.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/399M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

### Try out a basic prompt

In [ ]:
chat = [
    { "role": "user", "content": "Explain what is AI in 3 bullet points" },
]
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

In [ ]:
print(prompt)

<bos><start_of_turn>user
Explain what is AI in 3 bullet points<end_of_turn>
<start_of_turn>model



In [ ]:
model.device

device(type='cuda', index=0)

In [ ]:
tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

tensor([[     2,    106,   1645,    108,  74198,   1212,    603,  16481,    575,
         235248, 235304,  26353,   3782,    107,    108,    106,   2516,    108]])

In [ ]:
tokenizer.decode(tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")[0])

'<bos><start_of_turn>user\nExplain what is AI in 3 bullet points<end_of_turn>\n<start_of_turn>model\n'

In [ ]:
inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs.to(model.device),
                         max_new_tokens=150)
outputs

tensor([[     2,    106,   1645,    108,  74198,   1212,    603,  16481,    575,
         235248, 235304,  26353,   3782,    107,    108,    106,   2516,    108,
         235290,   5231,  11716,    591, 105460,  30515, 235275,    603,    573,
           7374,    576,    476,   6875,    689,   6479,    577,   3114,  13333,
            674,  15976,   2817,   3515,  17273, 235269,   1582,    685,   6044,
         235269,   3210, 235290,  60495, 235269,    578,   4530, 235290,  14577,
         116742,    108, 235290,   5231,  11716,   7177,  28514,    578,   1423,
            577,   3918,    578,   4771,   1163,   1069, 235269,   2346,   1855,
          34913,  73517, 116742,    108, 235290,   5231,  11716,    919,    573,
           5736,    577,  15693,    889,   1767,  12464,    576,   1167,   6672,
         235269,    774,  22908,    578,   5404,    577,  16216,    578,  15394,
         116742,      1]], device='cuda:0')

In [ ]:
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
Explain what is AI in 3 bullet points<end_of_turn>
<start_of_turn>model
- **AI (Artificial Intelligence) is the ability of a computer or machine to perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making.**
- **AI uses algorithms and data to learn and improve over time, without being explicitly programmed.**
- **AI has the potential to revolutionize many aspects of our lives, from healthcare and education to transportation and manufacturing.**<eos>


Remember to always refer to the [__documentation__](https://huggingface.co/docs/transformers/v4.18.0/en/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate) where all the arguments of the generation pipeline are mentioned in detail. Most notably:

- **max_length:** The maximum length of the sequence to be generated
- **max_new_tokens:** The maximum numbers of tokens to generate, ignore the current number of tokens. Use either max_new_tokens or max_length but not both, they serve the same purpose
- **do_sample:** Whether or not to use sampling. False means use greedy decoding i.e temperature=0
- **temperature:** Between 0 - 1, The value used to module the next token probabilities. Higher temperature means the results may vary and be more creative

In [ ]:
outputs = model.generate(input_ids=inputs.to(model.device),
                         max_new_tokens=150,
                         do_sample=True,
                         temperature=0
                         )
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
Explain what is AI in 3 bullet points<end_of_turn>
<start_of_turn>model
- **AI (Artificial Intelligence) is the ability of a computer or machine to perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making.**
- **AI uses algorithms and data to learn from experience and improve over time.**
- **AI has the potential to revolutionize many industries by automating tasks, improving efficiency, and solving complex problems.**<eos>


In [ ]:
print(prompt)

<bos><start_of_turn>user
Explain what is AI in 3 bullet points<end_of_turn>
<start_of_turn>model



### Pipelines make it easier to send prompts

You don't need to encode and decode your inputs and outputs everytime

In [ ]:
gemma_pipe = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="cuda",
)

In [ ]:
prompt

'<bos><start_of_turn>user\nExplain what is AI in 3 bullet points<end_of_turn>\n<start_of_turn>model\n'

In [ ]:
response = gemma_pipe(prompt,
                      max_new_tokens=150,
                      do_sample=True,
                      temperature=0.5,
                      return_full_text=False) # dont return back the input prompt, only show the response

In [ ]:
response

[{'generated_text': '- **AI (Artificial Intelligence) is the ability of a computer or machine to perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making.**\n- **AI uses algorithms and data to learn and improve over time, without being explicitly programmed.**\n- **AI has the potential to revolutionize many aspects of our lives, from healthcare and education to transportation and manufacturing.**'}]

In [ ]:
print(response[0]['generated_text'])

- **AI (Artificial Intelligence) is the ability of a computer or machine to perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making.**
- **AI uses algorithms and data to learn and improve over time, without being explicitly programmed.**
- **AI has the potential to revolutionize many aspects of our lives, from healthcare and education to transportation and manufacturing.**


In [ ]:
from IPython.display import display, Markdown

display(Markdown(response[0]['generated_text']))

- **AI (Artificial Intelligence) is the ability of a computer or machine to perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making.**
- **AI uses algorithms and data to learn and improve over time, without being explicitly programmed.**
- **AI has the potential to revolutionize many aspects of our lives, from healthcare and education to transportation and manufacturing.**

## Check how much GPU Memory the LLM Uses

Remember the Gemma-2B uses more than 5GB GPU memory

In [ ]:
!nvidia-smi

Sun Oct 20 12:37:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0              30W /  70W |   5431MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
gemma_pipe

## Prompting with Open-Source LLM

Now we will use our locally loaded LLM and try some tasks with prompting

### 1. Basic Q & A

In [ ]:
def create_gemma_prompt(prompt_text):
  chat = [
    { "role": "user", "content": prompt_text },
  ]
  prompt = tokenizer.apply_chat_template(chat, tokenize=False,
                                         add_generation_prompt=True)
  return prompt

In [ ]:
prompt_txt = "Can you explain what is mortgage?"
prompt = create_gemma_prompt(prompt_txt)
print(prompt)

<bos><start_of_turn>user
Can you explain what is mortgage?<end_of_turn>
<start_of_turn>model



In [ ]:
response = gemma_pipe(prompt,
                      max_new_tokens=1000,
                      do_sample=True,
                      temperature=0.5,
                      return_full_text=False)
response

[{'generated_text': "**Definition:**\n\nA mortgage is a legal loan that allows a borrower to purchase a home by putting down a down payment and taking out a loan for the remaining amount. The lender provides the funds for the purchase, and the borrower signs a mortgage agreement that outlines the terms of the loan, including the interest rate, loan amount, repayment period, and other conditions.\n\n**Key Features:**\n\n* **Loan type:** Mortgage loans are secured loans, meaning that the borrower provides collateral (the home) for the loan.\n* **Down payment:** A down payment is the initial payment that the borrower makes towards the purchase price of the home.\n* **Loan amount:** The amount of money that the borrower is eligible to borrow varies depending on the lender and the borrower's financial situation.\n* **Repayment period:** Mortgage loans typically have a fixed repayment period, which is the number of years over which the loan must be repaid.\n* **Interest rate:** The interest 

In [ ]:
print(response[0]['generated_text'])

**Definition:**

A mortgage is a legal loan that allows a borrower to purchase a home by putting down a down payment and taking out a loan for the remaining amount. The lender provides the funds for the purchase, and the borrower signs a mortgage agreement that outlines the terms of the loan, including the interest rate, loan amount, repayment period, and other conditions.

**Key Features:**

* **Loan type:** Mortgage loans are secured loans, meaning that the borrower provides collateral (the home) for the loan.
* **Down payment:** A down payment is the initial payment that the borrower makes towards the purchase price of the home.
* **Loan amount:** The amount of money that the borrower is eligible to borrow varies depending on the lender and the borrower's financial situation.
* **Repayment period:** Mortgage loans typically have a fixed repayment period, which is the number of years over which the loan must be repaid.
* **Interest rate:** The interest rate is the cost of borrowing m

In [ ]:
display(Markdown(response[0]['generated_text']))

**Definition:**

A mortgage is a legal loan that allows a borrower to purchase a home by putting down a down payment and taking out a loan for the remaining amount. The lender provides the funds for the purchase, and the borrower signs a mortgage agreement that outlines the terms of the loan, including the interest rate, loan amount, repayment period, and other conditions.

**Key Features:**

* **Loan type:** Mortgage loans are secured loans, meaning that the borrower provides collateral (the home) for the loan.
* **Down payment:** A down payment is the initial payment that the borrower makes towards the purchase price of the home.
* **Loan amount:** The amount of money that the borrower is eligible to borrow varies depending on the lender and the borrower's financial situation.
* **Repayment period:** Mortgage loans typically have a fixed repayment period, which is the number of years over which the loan must be repaid.
* **Interest rate:** The interest rate is the cost of borrowing money and is typically expressed as a percentage of the loan amount.
* **Loan terms:** Mortgage terms can range from a few months to 30 years, depending on the lender and the borrower's needs.

**Types of Mortgages:**

* **Fixed-rate mortgage:** The interest rate remains the same over the life of the loan.
* **Adjustable-rate mortgage:** The interest rate can fluctuate over the life of the loan.
* **FHA mortgage:** Backed by the Federal Housing Administration (FHA), which provides mortgage insurance to reduce the risk for lenders.
* **VA mortgage:** Backed by the Department of Veterans Affairs (VA), which provides mortgage loans to veterans and active-duty military members.
* **USDA mortgage:** Backed by the United States Department of Agriculture (USDA), which provides mortgage loans to individuals who live in rural areas.

**Benefits of a Mortgage:**

* **Homeownership:** Owning a home can provide stability, security, and a sense of community.
* **Equity:** Mortgage payments build equity in the home, which can be used as a down payment on future loans or as equity withdrawal.
* **Tax deductions:** Mortgage interest and property taxes are often tax-deductible.
* **Stability:** A mortgage provides financial stability and can help borrowers build wealth over time.

**Considerations:**

* **Financial responsibility:** Borrowers must be financially responsible and have a stable income to qualify for a mortgage.
* **Credit score:** A good credit score is often required to get the best mortgage terms.
* **Down payment:** The larger the down payment, the lower the monthly mortgage payment.
* **Interest rate:** The interest rate can have a significant impact on the overall cost of the loan.

### 2. Basic Sentiment Analysis

In [ ]:
reviews = [
    """I recently worked with this real estate company to purchase my first home,
    and the experience was outstanding. The agent was knowledgeable, patient, and incredibly responsive.
    They guided me through every step of the process, making what could have been a stressful
    experience very smooth and enjoyable.
    """,
    """This company's attention to detail and professionalism is second to none.
    Our agent went above and beyond to ensure we got the best deal possible.
    From the initial viewing to the final paperwork, everything was handled perfectly.
    We couldn’t be happier with our new property!
    """,
    """I was really let down by the lack of communication from this real estate company.
    It often took days to get a response, and I felt like I was always the last to know about
    updates on my property search. It made the whole buying process much more stressful than
    it needed to be.
    """,
    """My experience with this real estate company was frustrating. The agent seemed more interested
    in closing the deal quickly rather than finding what was best for me. I felt rushed and
    under-informed throughout the process, which has led to regrets about my purchase.
    """
]

In [ ]:
from tqdm import tqdm

responses = []

for review in tqdm(reviews):
  prompt_txt = f"""
  Act as a customer review analyst, given the following customer review text,
  do the following tasks:
  - Find the sentiment (positive, negative or neutral)
  - Extract max 5 key topics or phrases of the good or bad in the review

  Review Text:
  {review}
  """
  prompt = create_gemma_prompt(prompt_txt)
  llm_response = gemma_pipe(prompt,
                      max_new_tokens=150,
                      do_sample=False,
                      return_full_text=False)
  responses.append(llm_response[0]['generated_text'])

100%|██████████| 4/4 [00:17<00:00,  4.42s/it]


In [ ]:
responses

['**Sentiment Analysis:**\nThe tone of the review is positive. The reviewer is enthusiastic about their experience with the real estate company and describes the agent as knowledgeable, patient, and responsive.\n\n**Key Topics/Phrases:**\n- Outstanding experience\n- Knowledgeable agent\n- Patient\n- Incredibly responsive\n- Smooth and enjoyable experience',
 "**Sentiment Analysis:**\nThe overall sentiment of the review is positive. The reviewer expresses satisfaction with the company's attention to detail, professionalism, and overall service.\n\n**Key Topics/Phrases:**\n- Second to none\n- Ensure we got the best deal possible\n- Handled everything perfectly\n- New property\n- Perfect",
 '**Sentiment Analysis:**\n- The tone of the review is negative.\n\n**Key Topics/Phrases:**\n- Lack of communication\n- Days to get a response\n- Stressful buying process\n- Unresponsive agent\n- Lack of updates',
 "**Sentiment Analysis:**\nThe overall sentiment of the review is negative. The reviewer h

In [ ]:
for response in responses:
  print(response)

**Sentiment Analysis:**
The tone of the review is positive. The reviewer is enthusiastic about their experience with the real estate company and describes the agent as knowledgeable, patient, and responsive.

**Key Topics/Phrases:**
- Outstanding experience
- Knowledgeable agent
- Patient
- Incredibly responsive
- Smooth and enjoyable experience
**Sentiment Analysis:**
The overall sentiment of the review is positive. The reviewer expresses satisfaction with the company's attention to detail, professionalism, and overall service.

**Key Topics/Phrases:**
- Second to none
- Ensure we got the best deal possible
- Handled everything perfectly
- New property
- Perfect
**Sentiment Analysis:**
- The tone of the review is negative.

**Key Topics/Phrases:**
- Lack of communication
- Days to get a response
- Stressful buying process
- Unresponsive agent
- Lack of updates
**Sentiment Analysis:**
The overall sentiment of the review is negative. The reviewer had a negative experience with the real 

In [ ]:
for response in responses:
  display(Markdown(response))
  print()

**Sentiment Analysis:**
The tone of the review is positive. The reviewer is enthusiastic about their experience with the real estate company and describes the agent as knowledgeable, patient, and responsive.

**Key Topics/Phrases:**
- Outstanding experience
- Knowledgeable agent
- Patient
- Incredibly responsive
- Smooth and enjoyable experience

**Sentiment Analysis:**
The overall sentiment of the review is positive. The reviewer expresses satisfaction with the company's attention to detail, professionalism, and overall service.

**Key Topics/Phrases:**
- Second to none
- Ensure we got the best deal possible
- Handled everything perfectly
- New property
- Perfect

**Sentiment Analysis:**
- The tone of the review is negative.

**Key Topics/Phrases:**
- Lack of communication
- Days to get a response
- Stressful buying process
- Unresponsive agent
- Lack of updates

**Sentiment Analysis:**
The overall sentiment of the review is negative. The reviewer had a negative experience with the real estate company and felt rushed and under-informed.

**Key Topics/Phrases:**

- Frustrating experience
- Agent's focus on closing the deal quickly
- Feeling rushed and under-informed
- Regrets about the purchase
- Rushed and under-informed process

### 3. Content Generation based on topics

In [ ]:
prompt_txt = """Generate a bullet list of pros and cons of investing
                in commercial real estate during economic volatility.
                The list should include factors such as market potential,
                risk management, capital appreciation, and liquidity concerns
                """;
prompt = create_gemma_prompt(prompt_txt)
llm_response = gemma_pipe(prompt,
                      max_new_tokens=500,
                      do_sample=False,
                      return_full_text=False)

In [ ]:
llm_response

[{'generated_text': '**Pros of Investing in Commercial Real Estate During Economic Volatility:**\n\n- **Market Potential:** Commercial real estate remains a stable asset class, offering long-term investment opportunities despite economic fluctuations.\n- **Risk Management:** Diversification of investment portfolio with commercial real estate can mitigate risk exposure to other volatile markets.\n- **Capital Appreciation:** Over the long term, commercial real estate has historically shown steady appreciation, providing potential for financial growth.\n- **Liquidity:** Commercial real estate can be easily traded, providing liquidity for investors seeking to realize profits.\n- **Stable Rental Income:** Commercial properties can generate stable rental income, providing a consistent source of income.\n- **Tax Advantages:** Commercial real estate may offer tax deductions and other financial incentives.\n\n**Cons of Investing in Commercial Real Estate During Economic Volatility:**\n\n- **Mar

In [ ]:
print(llm_response[0]['generated_text'])

**Pros of Investing in Commercial Real Estate During Economic Volatility:**

- **Market Potential:** Commercial real estate remains a stable asset class, offering long-term investment opportunities despite economic fluctuations.
- **Risk Management:** Diversification of investment portfolio with commercial real estate can mitigate risk exposure to other volatile markets.
- **Capital Appreciation:** Over the long term, commercial real estate has historically shown steady appreciation, providing potential for financial growth.
- **Liquidity:** Commercial real estate can be easily traded, providing liquidity for investors seeking to realize profits.
- **Stable Rental Income:** Commercial properties can generate stable rental income, providing a consistent source of income.
- **Tax Advantages:** Commercial real estate may offer tax deductions and other financial incentives.

**Cons of Investing in Commercial Real Estate During Economic Volatility:**

- **Market Fluctuations:** Economic dow

In [ ]:
display(Markdown(llm_response[0]['generated_text']))

**Pros of Investing in Commercial Real Estate During Economic Volatility:**

- **Market Potential:** Commercial real estate remains a stable asset class, offering long-term investment opportunities despite economic fluctuations.
- **Risk Management:** Diversification of investment portfolio with commercial real estate can mitigate risk exposure to other volatile markets.
- **Capital Appreciation:** Over the long term, commercial real estate has historically shown steady appreciation, providing potential for financial growth.
- **Liquidity:** Commercial real estate can be easily traded, providing liquidity for investors seeking to realize profits.
- **Stable Rental Income:** Commercial properties can generate stable rental income, providing a consistent source of income.
- **Tax Advantages:** Commercial real estate may offer tax deductions and other financial incentives.

**Cons of Investing in Commercial Real Estate During Economic Volatility:**

- **Market Fluctuations:** Economic downturns can lead to decreased demand for commercial real estate, resulting in market downturns and potential losses.
- **Interest Rate Risk:** Rising interest rates can make it more expensive to borrow money for commercial real estate investments, potentially impacting returns.
- **Credit Risk:** Commercial real estate investments can be vulnerable to credit defaults, especially in times of economic stress.
- **Maintenance and Repairs:** Maintaining and repairing commercial properties can be costly, especially during economic downturns when budgets may be tight.
- **Competition:** The commercial real estate market can be competitive, with many investors vying for the same properties.
- **Legal and Regulatory Risks:** Commercial real estate investments can be subject to various legal and regulatory complexities.

### 4. Report Summarization

In [ ]:
report = """
Generative AI is a type of artificial intelligence technology that can produce various types of content, including text, imagery, audio and synthetic data. The recent buzz around generative AI has been driven by the simplicity of new user interfaces for creating high-quality text, graphics and videos in a matter of seconds.
The technology, it should be noted, is not brand-new. Generative AI was introduced in the 1960s in chatbots. But it was not until 2014, with the introduction of generative adversarial networks, or GANs -- a type of machine learning algorithm -- that generative AI could create convincingly authentic images, videos and audio of real people.
On the one hand, this newfound capability has opened up opportunities that include better movie dubbing and rich educational content. It also unlocked concerns about deepfakes -- digitally forged images or videos -- and harmful cybersecurity attacks on businesses, including nefarious requests that realistically mimic an employee's boss.
Two additional recent advances that will be discussed in more detail below have played a critical part in generative AI going mainstream: transformers and the breakthrough language models they enabled. Transformers are a type of machine learning that made it possible for researchers to train ever-larger models without having to label all of the data in advance. New models could thus be trained on billions of pages of text, resulting in answers with more depth. In addition, transformers unlocked a new notion called attention that enabled models to track the connections between words across pages, chapters and books rather than just in individual sentences. And not just words: Transformers could also use their ability to track connections to analyze code, proteins, chemicals and DNA.
The rapid advances in so-called large language models (LLMs) -- i.e., models with billions or even trillions of parameters -- have opened a new era in which generative AI models can write engaging text, paint photorealistic images and even create somewhat entertaining sitcoms on the fly. Moreover, innovations in multimodal AI enable teams to generate content across multiple types of media, including text, graphics and video. This is the basis for tools like Dall-E that automatically create images from a text description or generate text captions from images.
These breakthroughs notwithstanding, we are still in the early days of using generative AI to create readable text and photorealistic stylized graphics. Early implementations have had issues with accuracy and bias, as well as being prone to hallucinations and spitting back weird answers. Still, progress thus far indicates that the inherent capabilities of this generative AI could fundamentally change enterprise technology how businesses operate. Going forward, this technology could help write code, design new drugs, develop products, redesign business processes and transform supply chains.
"""

prompt_txt = f"""
Summarize the following report delimited by triple backticks on Generative AI in max 5 lines

Report:
```{report}```
"""

prompt = create_gemma_prompt(prompt_txt)

llm_response = gemma_pipe(prompt,
                      max_new_tokens=500,
                      do_sample=False,
                      return_full_text=False)

In [ ]:
print(llm_response[0]['generated_text'])

- Generative AI can produce various types of content, including text, imagery, audio, and synthetic data.
- It uses techniques like transformers and large language models to create realistic images, videos, and text.
- Generative AI has both opportunities and concerns, such as deepfakes and cybersecurity risks.
- Recent advances have made generative AI more powerful and versatile, enabling it to write text, create images, and generate content across multiple media.
- Despite its potential, generative AI is still in its early stages and requires further development to address accuracy and bias issues.


In [ ]:
display(Markdown(llm_response[0]['generated_text']))

- Generative AI can produce various types of content, including text, imagery, audio, and synthetic data.
- It uses techniques like transformers and large language models to create realistic images, videos, and text.
- Generative AI has both opportunities and concerns, such as deepfakes and cybersecurity risks.
- Recent advances have made generative AI more powerful and versatile, enabling it to write text, create images, and generate content across multiple media.
- Despite its potential, generative AI is still in its early stages and requires further development to address accuracy and bias issues.

### 5. Context-based QA

In [ ]:
report = """
Three quarters (77%) of the population saw an increase in their regular outgoings over the past year,
according to findings from our recent consumer survey. In contrast, just over half (54%) of respondents
had an increase in their salary, which suggests that the burden of costs outweighing income remains for
most. In total, across the 2,500 people surveyed, the increase in outgoings was 18%, three times higher
than the 6% increase in income.

Despite this, the findings of our survey suggest we have reached a plateau. Looking at savings,
for example, the share of people who expect to make regular savings this year is just over 70%,
broadly similar to last year. Over half of those saving plan to use some of the funds for residential
property. A third are saving for a deposit, and a further 20% for an investment property or second home.

But for some, their plans are being pushed back. 9% of respondents stated they had planned to purchase
a new home this year but have now changed their mind. While for many the deposit may be an issue,
the other driving factor remains the cost of the mortgage, which has been steadily rising the last
few years. For those that currently own a property, the survey showed that in the last year,
the average mortgage payment has increased from £668.51 to £748.94, or 12%."""


In [ ]:
question = """
How much has the average mortage payment increased in the last year?
"""

prompt_txt = f"""
Using the following context information below please answer the following question
to the best of your ability
Context:
{report}
Question:
{question}
"""

prompt = create_gemma_prompt(prompt_txt)

llm_response = gemma_pipe(prompt,
                      max_new_tokens=500,
                      do_sample=False,
                      return_full_text=False)

In [ ]:
display(Markdown(llm_response[0]['generated_text']))

According to the context, the average mortgage payment has increased from £668.51 to £748.94, or 12%.

In [ ]:
question = """
What percentage of people had an increase in salary last year? Show the answer just as a number.
"""

prompt_txt = f"""
Using the following context information below please answer the following question
to the best of your ability
Context:
{report}
Question:
{question}
"""

prompt = create_gemma_prompt(prompt_txt)

llm_response = gemma_pipe(prompt,
                      max_new_tokens=500,
                      do_sample=False,
                      return_full_text=False)

In [ ]:
display(Markdown(llm_response[0]['generated_text']))

54%